In [18]:
import pandas as pd
import numpy as np
import requests
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime


In [19]:
wallets = pd.read_csv("wallets.csv")  # after downloading from Google Sheet


In [43]:
import random

def get_wallet_data(wallet_address):
    total_supply = random.uniform(500, 5000)
    total_borrow = random.uniform(0, total_supply)  # borrow ≤ supply
    borrow_supply_ratio = total_borrow / total_supply if total_supply > 0 else 0

    return {
        "wallet_id": wallet_address,
        "total_supply": total_supply,
        "total_borrow": total_borrow,
        "borrow_supply_ratio": borrow_supply_ratio
    }


In [44]:
# Example: loading wallets from Google Sheet
url = "https://docs.google.com/spreadsheets/d/1ZzaeMgNYnxvriYYpe8PE7uMEblTI0GV5GIVUnsP-sBs/export?format=csv"
wallet_df = pd.read_csv(url)


In [45]:
wallet_features = []

for wallet in wallet_df['wallet_id']:
    data = get_wallet_data(wallet)
    wallet_features.append(data)

features_df = pd.DataFrame(wallet_features)


In [46]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
features_to_normalize = ['total_supply', 'total_borrow', 'borrow_supply_ratio']
features_df[features_to_normalize] = scaler.fit_transform(features_df[features_to_normalize])


In [47]:
features_df['score_raw'] = (
    0.4 * features_df['borrow_supply_ratio'] +
    0.3 * features_df['total_borrow'] +
    0.3 * features_df['total_supply']
)

# Normalize to 0–1000
score_range = features_df['score_raw'].max() - features_df['score_raw'].min()

if score_range == 0:
    features_df['score'] = 500
else:
    features_df['score'] = (
        (features_df['score_raw'] - features_df['score_raw'].min()) / score_range
    ) * 1000


In [48]:
print(features_df[['wallet_id', 'score']].head())
print(features_df['score'].isnull().sum())

                                    wallet_id       score
0  0x0039f22efb07a647557c7c5d17854cfd6d489ef3   33.122744
1  0x06b51c6882b27cb05e712185531c1f74996dd988  230.653943
2  0x0795732aacc448030ef374374eaae57d2965c16c  378.544501
3  0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9  556.148394
4  0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae  100.934874
0


In [50]:
features_df['score'] = features_df['score'].round().astype(int)
features_df = features_df.sort_values(by='score', ascending=False)

In [51]:
features_df[['wallet_id', 'score']].to_csv("wallet_scores.csv", index=False)


## Data Collection

- Wallets were loaded from the provided Google Sheet.
- For each wallet, mock data was generated using a function.
- In production, this would connect to Compound's subgraph or REST API.

## Feature Selection

- **total_supply**: Total value supplied to the protocol
- **total_borrow**: Total borrowed amount
- **borrow_supply_ratio**: Indicator of leverage risk

## Risk Scoring Logic

The score is calculated using a weighted sum:

- 40%: borrow/supply ratio (higher ratio = more risk)
- 30%: total borrow (larger borrow = more exposure)
- 30%: total supply (more supply = safer profile)

All scores are normalized to a 0–1000 scale.


In [55]:
print(features_df[['wallet_id', 'score']].head(10))


                                     wallet_id  score
13  0x1c1b30ca93ef57452d53885d97a74f61daf2bf4f   1000
7   0x124853fecb522c57d9bd5c21231058696ca6d596    961
31  0x4db0a72edb5ea6c55df929f76e7d5bb14e389860    955
28  0x4839e666e2baf12a51bf004392b35972eeddeabf    921
14  0x1e43dacdcf863676a6bec8f7d6896d6252fac669    876
75  0xa7f3c74f0255796fd5d3ddcf88db769f7a6bf46a    871
56  0x7e3eab408b9c76a13305ef34606f17c16f7b33cc    817
62  0x880a0af12da55df1197f41697c1a1b61670ed410    806
42  0x612a3500559be7be7703de6dc397afb541a16f7f    787
52  0x7b4636320daa0bc055368a4f9b9d01bd8ac51877    774
